cGAN으로 생성 제어하기 

- GAN이 더욱 쓸모 있으려면 사용자가 원하는 이미지를 생성하는 기능 제공해야 한다. 
- 생성과정에서 생성하고자 하는 레이블 정보를 추가로 넣어 원하는 이미지가 나오게끔 모델 수정
- 앞의 생성 이미지는 어러 종류의 패션 아이템을 무작위 벡터를 입력받아 출력한 것이다. 
- 조건부 GAN (cGAN)의 생성자는 학습 과정에서 생성하고픈 아이템의 종류를 입력받아야 한다. 
  - 이를 구현하는 방법은 생성자와 판별자의 입력에 레이블 정보를 이어 붙이는 것이다. 
  

In [1]:
#라이브러리 임포트
import os 
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import numpy as np

#하이퍼 파라미터 설정

EPOCHS = 50
BATCH_SIZE = 100
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')
print('다음 장치 이용 :', DEVICE)

#데이터셋 로드 

trainset = datasets.FashionMNIST(
    './.data',
    train=True,
    download=True,
    transform=transforms.Compose([
       transforms.ToTensor(),
       transforms.Normalize((0.5,), (0.5,))
    ])
)
train_loader = torch.utils.data.DataLoader(
    dataset     = trainset,
    batch_size  = BATCH_SIZE,
    shuffle     = True
)

다음 장치 이용 : cpu


In [6]:
#생성자 
#이번 예제는 무작위 텐서(z)의 크기를 100개로 지정

class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        #연속적인 값이 학습에 유용하여 배치 x1 크기의 레이블 텐서를 받아 배치 x10의 연속적인 텐서로 전환 
        self.embed = nn.Embedding(10,10)
        
        #생성자의 첫 입력이 110개인 이유는 텐서크기100, 레이블값10이 더해진 값
        self.model = nn.Sequential(nn.Linear(110,256),
                                  nn.LeakyReLU(0.2, inplace = True), #inplace는 입력을 복사하지 않고 바로 조작설정 변수
                                  nn.Linear(256, 512),
                                  nn.LeakyReLU(0.2, inplace = True),
                                  nn.Linear(512, 1024),
                                  nn.LeakyReLU(0.2, inplace = True),
                                  nn.Linear(1024, 784),
                                  nn.Tanh())
        
    def forward(self, z, labels):
        c = self.embed(labels)
        x = torch.cat([z, c], 1) #두 벡터를 이어 붙이는 연산, 무작위 벡터와 클래스 레이블을 이어붙이고 생성자에 입력 
        return self.model(x)

In [9]:
#판별자
#판별자 역시 레이블 정보를 받는다. 
#생성자에서 이미지를 만들때 쓴 레이블 정보를 입력받아 '레이블이 주어진 경우의 가짜인 확률과 진짜인 확률을 추정한다'라고 생각하면된다. 

class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.embed = nn.Embedding(10,10)
        
        #판별자에도 생성자의 출력값(이미지크기)에 10을 더해준다
        #1024노드 출력계층과, 성능을 위해 각 계층 사이의 드롭아웃 계층을 추가 
        #마지막은 Sigmoid함수를 거쳐 참, 거짓을 뜻하는 0~1사이 값 반환 
        self.model = nn.Sequential(nn.Linear(794, 1024),
                                  nn.LeakyReLU(0.2, inplace = True),
                                  nn.Dropout(0.3),
                                  nn.Linear(1024, 512),
                                  nn.LeakyReLU(0.2, inplace = True),
                                  nn.Dropout(0.3),
                                  nn.Linear(512, 256),
                                  nn.LeakyReLU(0.2, inplace = True),
                                  nn.Dropout(0.3),
                                  nn.Linear(256,1),
                                  nn.Sigmoid())
        
    def forward(self, x, labels):
        c = self.embed(labels)
        x = torch.cat([x, c], 1)
        return self.model(x)

In [10]:
#모델의 인스턴스를 만들고, 모델의 가중치를 지정 장치로 보낸다. 
D = Discriminator().to(DEVICE)
G = Generator().to(DEVICE)

#이진교차 엔트로피 오차함수, 생성자 판별자를 최적화할 Adam모듈 설정 
criterion = nn.BCELoss()
d_optimizer = optim.Adam(D.parameters(), lr = 0.0002)
g_optimizer = optim.Adam(G.parameters(), lr = 0.0002)

In [12]:
#데이터 로더(trian_loader)의 두 번째 반환값도 사용할 것이므로 레이블 표기
#일반 GAN 예제와 같이 진짜와 가짜 레이블 만든다. 

total_step = len(train_loader)
for epoch in range(EPOCHS):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(BATCH_SIZE, -1).to(DEVICE)
        
        #진짜, 가짜 레이블 생성
        real_labels = torch.ones(BATCH_SIZE, 1).to(DEVICE)
        fake_labels = torch.zeros(BATCH_SIZE, 1).to(DEVICE)
        
        #판별자가 진짜 이미지를 인식하는 오차를 계산(데이터셋의 레이블을 입력해 판별자가 이미지와 레이블의 관계를 학습하게 한다. )
        labels = labels.to(DEVICE)
        outputs = D(images, labels)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs 
        
        #무작위 텐서 생성 - 0~10사이의 값을 가친 배치x1 크기의 텐서를 만들고 생성자에 입력해 관계성 학습
        z = torch.randn(BATCH_SIZE, 100).to(DEVICE)
        g_label = torch.randint(0,10, (BATCH_SIZE,)).to(DEVICE)
        fake_images=G(z, g_label)
        
        #가짜 이미지를 판별자에게 입력하고, 생성자가 이용한 레이블과 결과물 이미지를 보고 가짜라고 인식하는 오차 계산
        ouputs = D(fake_images, g_label)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        #진짜, 가짜, 각 레이블을 보고 계산한 판별자의 총오차 계산하고 역전파 알고리즘으로 판별자의 학습 진행
        d_loss = d_loss_real + d_loss_fake
        
        d_optimizer.zero_grad()#기울기 초기화
        g_optimizer.zero_grad()
        d_loss.backward()#역전파 
        d_optimizer.step
        
        #생성자도 z와 g_label로 이미지를 생성하고 판별자를 속이는지에 대한 오차 계산
        fake_images = G(z, g_label)
        outputs = D(fake_images, g_label)
        g_loss = criterion(outputs, real_labels)
        
        #역전파 알고리즘으로 생성자 모델의 학습진행
        d_optimizer.zero_grad()
        g_optimizer.zero_grad()
        g_loss.backward()
        g_optimizer.step()
        
    #학습진행
    #딥러닝 환경이 원활하지 않아 epoch를 50회만 진행하여 성능이 떨어짐 
    #판별자의 오차 = d_loss // 생성자의 오차 = g_loss // 판별자가 진짜를 진짜로 인식 정확도 = D(x) // 가짜를 진짜로 인식한 정확도 D(G(z))를 통해 학습진행확인
    print('EPOCH : {}/{}  //  d_loss  :  {}  // g_loss :  {}  // D(x) : {}  // D(G(z)) :  {}'.format(
    epoch, EPOCHS, d_loss.item(), g_loss.item(), real_score.mean().item(), fake_score.mean().item()))

EPOCH : 0/50  //  d_loss  :  1.389394998550415  // g_loss :  0.39439141750335693  // D(x) : 0.4751477837562561  // D(G(z)) :  0.4751477837562561
EPOCH : 1/50  //  d_loss  :  1.388519525527954  // g_loss :  0.4014612138271332  // D(x) : 0.47970935702323914  // D(G(z)) :  0.47970935702323914
EPOCH : 2/50  //  d_loss  :  1.3892841339111328  // g_loss :  0.3967306911945343  // D(x) : 0.4762822687625885  // D(G(z)) :  0.4762822687625885
EPOCH : 3/50  //  d_loss  :  1.3892929553985596  // g_loss :  0.3967667818069458  // D(x) : 0.4758261442184448  // D(G(z)) :  0.4758261442184448
EPOCH : 4/50  //  d_loss  :  1.3893404006958008  // g_loss :  0.3965621888637543  // D(x) : 0.4770349860191345  // D(G(z)) :  0.4770349860191345
EPOCH : 5/50  //  d_loss  :  1.389049768447876  // g_loss :  0.3941905200481415  // D(x) : 0.47767341136932373  // D(G(z)) :  0.47767341136932373
EPOCH : 6/50  //  d_loss  :  1.3887007236480713  // g_loss :  0.3961203396320343  // D(x) : 0.47898852825164795  // D(G(z)) :  0

In [ ]:
#결과물 시각화
#만들고 싶은 아이템 생성하고 시각화 하기 
# 0 = 티셔츠 / 1 = 바지 / 2 = 스웨터 / 3 = 드레스 / 4 = 코트 / 5 = 샌들 / 6 = 셔츠 / 7 = 신발 / 8 = 가방 / 9 = 부츠

item_number = 4
z = torch.randn(1, 100).to(DEVICE) #배치크기 1

#torch.full은 새로운 텐서를 만드는 텐서 - 텐서 크기, 텐서 원소들을 초기화 할 값을 받아 아이템번호를 포함한 g_label이라는 1차원 텐서 생성
g_label = torch.full((1, ), item_number, dtype = torch.long).to(DEVICE)

#생성자에 무작위 텐서와 g_label을 입력하여 이미지생성
sample_images = G(z, g_label)

#시각화를 위한 넘파이 행렬로 변환 
sample_images_img = np.reshape(sample_images.data.cpu().numpy()[0],(28,28))

plt.imshow(sample_images_img, cmap = 'gray')
plt.show()